In [1]:
import gurobipy as gp
import pandas as pd
from code_map import final_markets, new_meters, utils, data_handling, timeframes
import numpy as np
from datetime import datetime, timedelta



In [3]:
timeframe = timeframes.half_month
L, M, F, H, freq_data, power_meter_dict, consumption_data = utils.get_all_sets(timeframe)


/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/UIB/Master/master-kode/code_map/utils.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


In [4]:
L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m = utils.get_parameters(L,M,H)

In [5]:
markets_name_dict = {market.name: market for market in M}
market_names = list(markets_name_dict.keys())

In [6]:
dominant_directions = [utils.get_dominant_direction(freq_data, hour) for hour in H]
Ir_hlm, Ia_hlm, Va_hm = utils.get_income_dictionaries(H, M, L, dominant_directions, Fu_h_l, Fd_h_l, P_h_m, Vp_h_m, F, markets_name_dict, timeframe=timeframe)


Regulation Up Activated


/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/UIB/Master/master-kode/code_map/utils.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


Regulation Down Activated


/Users/sandermeland/Documents/Dokumenter – Sanders MacBook Air/UIB/Master/master-kode/code_map/utils.py:132: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(by = "Time", inplace = True)


In [7]:
compatible_list = utils.get_compatibility_list(H=H, L=L, M=M)


In [ ]:
data_handling.save_collections(L, M, F, H, freq_data, power_meter_dict, consumption_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m, markets_name_dict, market_names, dominant_directions, Ir_hlm, Ia_hlm, Va_hm, compatible_list)

In [14]:
def find_previous_weekday(d, weekday):
    days_behind = d.weekday() - weekday
    if days_behind < 0: # Target day is in the previous week
        days_behind += 7
    return d - timedelta(days=days_behind)

# Function to calculate the deadlines for a given input timestamp
def calculate_deadlines_for_input(input_timestamp):
    deadlines = {}

    # RKOM uke: Friday the week before if on a weekday, otherwise Thursday the same week
    if input_timestamp.weekday() < 5:  # Weekday
        deadlines['RKOM uke Weekday'] = find_previous_weekday(input_timestamp - timedelta(days=7), 4)  # Previous Friday
    else:  # Weekend
        deadlines['RKOM uke Weekend'] = find_previous_weekday(input_timestamp, 3)  # Current Thursday

    # FCR D-2: Two days before the input time at 17:30
    deadlines['FCR D-2'] = datetime.combine(input_timestamp.date() - timedelta(days=2), datetime.strptime('17:30', '%H:%M').time())

    # aFRR CM: One day before the input time at 07:30
    deadlines['aFRR'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('07:30', '%H:%M').time())

    # FCR D-1: One day before the input time at 18:00
    deadlines['FCR D-1'] = datetime.combine(input_timestamp.date() - timedelta(days=1), datetime.strptime('18:00', '%H:%M').time())

    # RK: 45 minutes before the input time
    deadlines['RK'] = input_timestamp - timedelta(minutes=45)

    return deadlines

In [15]:
# Example usage of the function
example_input_timestamp =H[0]  # Example: 13th December 2023, 15:00
print(f"Input timestamp : {H[0]}")
deadlines = calculate_deadlines_for_input(example_input_timestamp)
for market, deadline in deadlines.items():
    print(f"{market}: {deadline.strftime('%Y-%m-%d %H:%M')}")

Input timestamp : 2023-06-14 00:00:00+02:00
RKOM uke Weekday: 2023-06-02 00:00
FCR D-2: 2023-06-12 17:30
aFRR: 2023-06-13 07:30
FCR D-1: 2023-06-13 18:00
RK: 2023-06-13 23:15


In [24]:
weekly_timeframe = utils.get_timestampstimeframes.one_week

In [28]:
data_handling.save_collections(L = L, M = M, F = F, H = H, freq_data= freq_data, power_meter_dict= power_meter_dict, consumption_data= consumption_data, L_u= L_u, L_d= L_d, Fu_h_l= Fu_h_l, Fd_h_l= Fd_h_l, R_h_l= R_h_l, P_h_m= P_h_m, Vp_h_m= Vp_h_m, Vm_m= Vm_m, R_m= R_m, dominant_directions= dominant_directions, Ir_hlm= Ir_hlm, Ia_hlm= Ia_hlm, Va_hm= Va_hm, compatible_list= compatible_list)

: 

In [22]:
def make_bids_for_one_week(weekly_timeframe, L,M,H, freq_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m):
    
    for h, hour in enumerate(list(weekly_timeframe)): #19-26.06.2023
        deadlines = calculate_deadlines_for_input(hour)
        print(deadlines)
        
        
    
    return None

In [23]:
make_bids_for_one_week(weekly_timeframe, L,M,H, freq_data, L_u, L_d, Fu_h_l, Fd_h_l, R_h_l, P_h_m, Vp_h_m, Vm_m, R_m)

TypeError: 'TimeFrame' object is not iterable